In [1]:
import re
from collections import defaultdict
from ipywidgets import Button, HBox, VBox, Layout, widgets
from IPython.display import display

In [2]:
class IF:
    """text"""
    
    def __init__(self, path, current = None, future = None):
        """text"""
        self.end = False
        if path: # first time
            self.choice = None
            self._create_tree(path)
        else: # recursive
            self._create_branch(current, future)
                            
    def _create_tree(self, path):
        """"""
        f = open(path)
        raw = f.read().split('State 0')[1:]
        
        next_states = defaultdict(lambda: [])
        
        for item in raw:
            path = item.split('State')
            path = self._clean_path(path)
            
            self.reactions = tuple(path[0])
            path = path[1:]
            
            next_states[tuple(path[0])].append(path[1:])
        
        self.children = [IF(None, state, next_states[state]) for state in next_states]
    
    def _clean_path(self, raw_path):
        """"""
        path = []
        for item in raw_path:
            item = item.replace('\n','')
            item = item.replace(':', '')
            item = ''.join([i for i in item if not i.isdigit()])
            item = item.strip(' ')
            item = item.split(' ')
            token = re.compile("\w+\(.*\)")
            for idx in range(len(item)):
                if token.match(item[idx]):
                    item[idx] = token.match(item[idx]).group()
                else:
                    item[idx] = None
            item = list(filter(None, item))
            path.append(item)
        return path
    
    def _create_branch(self, current, future):
        """"""        
        choice = re.compile('decision\(.*\)')
        self.choice = None
        self.reactions = []
        for item in current:
            if choice.match(item):
                self.choice = choice.match(item).group()
            else:
                self.reactions.append(item)
        
        next_states = defaultdict(lambda: [])   
            
        if future:
            for item in future:
                if len(item) > 1:
                    next_states[tuple(item[0])].append(item[1:])
                else:
                    next_states[tuple(item[0])] = None
            self.children = [IF(None, state, next_states[state]) for state in next_states]
        else:
            self.end = True
            self.children = None
            
    def play(self):
        for item in self.reactions:
            print(str(item))
        choices = True
        if not self.children:
            choices = False
        else:
            for child in self.children:
                if not child.choice:
                    choices = False
        if choices == True:
            buttons = [Button(description=str(child.choice), 
                              layout=Layout(width='100%'))
                        for child in self.children]
            display(widgets.VBox(buttons))
            for idx in range(len(self.children)):
                buttons[idx].on_click(self.children[idx]._choose)
        else:
            #for item in self.reactions:
            #    print(str(item))
            if self.children:
                assert(len(self.children) == 1)
                self.children[0].play()
        
    def _choose(self, b):
        #for item in self.reactions:
        #    print(str(item))
        self.play()       
        

# Testing

In [25]:
heist = IF('heist-results.txt')
# when imin=18: 5108
# ~322,000 models, 473 seconds, 15 choice steps

In [26]:
heist.play()

setting(museum)
story(start)
is(character(thief),place(museum))
story(rising)
is(character(thief),place(museum))


A Jupyter Widget

is(character(thief),place(museum))
occurs(character(thief),verb(find,trans(object(painting))),act(rising))
occurs(character(thief),verb(search,trans(object(painting))),act(rising))


A Jupyter Widget

is(character(thief),place(museum))
occurs(character(guard),verb(find,trans(character(thief))),act(rising))
occurs(character(thief),verb(find,trans(character(guard))),act(rising))
occurs(character(thief),verb(search,trans(character(guard))),act(rising))


...

has(character(thief),object(painting))
is(character(thief),adj(hurt))
is(character(thief),place(museum))
occurs(character(guard),verb(fight,trans(character(thief))),act(falling))
occurs(character(thief),verb(find,trans(object(key))),act(falling))
occurs(character(thief),verb(search,trans(object(key))),act(falling))


A Jupyter Widget

has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),adj(hurt))
is(character(thief),place(museum))
occurs(character(thief),verb(take,trans(object(key))),act(falling))


A Jupyter Widget

has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),adj(hurt))
is(character(thief),place(museum))
occurs(character(thief),verb(escape,none),act(falling))
story(end)
has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),place(museum))


In [23]:
lost_child = IF('results.txt')

In [24]:
lost_child.play()

setting(museum)
story(start)
is(character(thief),place(museum))
story(rising)
is(character(thief),place(museum))


A Jupyter Widget

is(character(thief),place(museum))
occurs(character(guard),verb(find,trans(character(thief))),act(rising))
occurs(character(thief),verb(find,trans(character(guard))),act(rising))
occurs(character(thief),verb(search,trans(character(guard))),act(rising))


A Jupyter Widget

is(character(thief),place(museum))
occurs(character(thief),verb(lie,trans(character(guard))),act(rising))
occurs(character(thief),verb(trick,trans(character(guard))),act(rising))


...

has(character(thief),object(painting))
is(character(thief),place(museum))
occurs(character(thief),verb(find,trans(object(key))),act(falling))
occurs(character(thief),verb(search,trans(object(key))),act(falling))


A Jupyter Widget

has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),place(museum))
occurs(character(thief),verb(take,trans(object(key))),act(falling))


A Jupyter Widget

has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),place(museum))
occurs(character(thief),verb(escape,none),act(falling))
story(end)
has(character(thief),object(key))
has(character(thief),object(painting))
is(character(thief),place(museum))
